# Imports

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from snmf import SNMF
import EDXS_model
import h5py
# hyperspy is the commonly used library to handle spectrum images in the electron microscopy community
import hyperspy.api as hs
import utils as u

# Load the data

In [2]:
# filename="Data/71GPa_subsolidus"
filename = "Data/aspim037_N100_2ptcls_brstlg"

S=hs.load(filename+".hspy")
X=S.data

# This part of the spectrum image contains only pure spectra from phase 0
# This kind of area is often available in experimental datasets
X_part=S.inav[60:,:].data

# Performance assessment functions

These functions are used to compare the endmembers determined by SNMF and the ground truth

In [3]:
# This function will find the best matching endmember for each true spectrum. This is useful since the A and P matrice are initialized at random. 

# This function works but can probably greatly improved
def find_min_angle (list_true_vectors,list_algo_vectors) :
    # This function calculates all the possible angles between endmembers and true spectra
    # For each true spectrum a best matching endmember is found
    # The function returns the angles of the corresponding pairs
    copy_algo_vectors=list_algo_vectors.copy()
    size=list_algo_vectors[0].shape
    ordered_angles=[]
    for i in range(len(list_true_vectors)) :
        list_angles=[]
        for j in range(len(list_algo_vectors)) :
            list_angles.append(u.Functions.spectral_angle(list_true_vectors[i],list_algo_vectors[j]))
        ind_min=np.argmin(np.array(list_angles))
        list_algo_vectors[ind_min]=1e28*np.ones(size)
        ordered_angles.append(u.Functions.spectral_angle(list_true_vectors[i],copy_algo_vectors[ind_min]))
    return ordered_angles

# This function works but can probably greatly improved
def find_min_MSE (list_true_maps,list_algo_maps) :
    # This function calculates all the possible MSE between abundances and true maps
    # For each true map a best matching abundance is found
    # The function returns the MSE of the corresponding pairs
    copy_algo_maps=list_algo_maps.copy()
    size=list_algo_maps[0].shape
    ordered_maps=[]
    for i in range(len(list_true_maps)) :
        list_maps=[]
        for j in range(len(list_algo_maps)) :
            list_maps.append(u.Functions.MSE_map(list_true_maps[i],list_algo_maps[j]))
        ind_min=np.argmin(np.array(list_maps))
        list_algo_maps[ind_min]=1e28*np.ones(size)
        ordered_maps.append(u.Functions.MSE_map(list_true_maps[i],copy_algo_maps[ind_min]))
    return ordered_maps

# This function gives the residuals between the model determined by snmf and the data that were fitted
def residuals (data,model) :
    X_sum=data.sum(axis=0).sum(axis=0)
    model_sum=model.get_phase_map(0).sum()*model.get_phase_spectrum(0)+model.get_phase_map(1).sum()*model.get_phase_spectrum(1)+model.get_phase_map(2).sum()*model.get_phase_spectrum(2)
    return X_sum-model_sum

# Parameters 

In [4]:
# True bremsstrahlung parameters
brstlg_pars = {"c0" : 4.8935e-05,"c1" : 1464.19810, "c2" : 0.04216872, "b0" : 0.15910789, "b1" : -0.00773158, "b2" : 8.7417e-04}
# brstlg_pars = {
#                 "c0"  :  8.0192e-08,
#                 "c1"  :  380.523471	,
#                 "c2"  :  2.7332e-11,
#                 "b0"  :  0.09002422,
#                 "b1"  :  -0.0162849,
#                 "b2"  :  0.00193291
#                 }


# SNMF parameters
tol = 1e-4
max_iter = 50000
b_tol = 1e-1
mu_sparse = 0.0
eps_sparse = 1.0
phases = 3
em = EDXS_model.EDXS_Model("Data/simple_xrays_threshold.json",brstlg_pars = brstlg_pars,e_offset=S.axes_manager[2].offset,e_scale=S.axes_manager[2].scale,e_size=S.axes_manager[2].size)
em.generate_g_matr([8,13,14,12,26,29,31,72,71,62,60,92,20],brstlg = False)
# em.generate_g_matr([12,13,14,20,26,29,31,34,40,60,62,71,92],brstlg = True)

# Loading of ground truth
true_spectra=[]
true_maps=[]
true_spectra.append(np.genfromtxt(filename+"spectrum_p0"))
true_spectra.append(np.genfromtxt(filename+"spectrum_p1"))
true_spectra.append(np.genfromtxt(filename+"spectrum_p2"))
true_maps.append(np.load(filename+"map_p0.npy"))
true_maps.append(np.load(filename+"map_p1.npy"))
true_maps.append(np.load(filename+"map_p2.npy"))

# # If mu_sparse !=0 a good initialization of the first phase is required, it can be done using the spectrum below
init_matrix=np.average(X_part,axis=(0,1))

# SNMF

In [5]:
# Creation of an SNMF object with the parameters above
mdl = SNMF(max_iter = max_iter, tol = tol, b_tol = b_tol, mu_sparse=mu_sparse, eps_sparse = eps_sparse, num_phases=phases,edxs_model=em, brstlg_pars = brstlg_pars, init_spectrum = init_matrix)

In [6]:
mdl.fit(X,eval_print=True)

04 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 4.854860544204712 secondsafter b 0.0
Finished iteration 12105 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 3.6488592624664307 secondsafter b 2.3283064365386963e-10
Finished iteration 12106 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 3.6249537467956543 secondsafter b 0.0
Finished iteration 12107 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 3.521897315979004 secondsafter b 0.0
Finished iteration 12108 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 3.8777036666870117 secondsafter b 0.0
Finished iteration 12109 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 3.507946491241455 secondsafter b 0.0
Finished iteration 12110 of maximal 50000 function value decreased by: 0.00018596858717501163 taking: 3.500734329223633 secondsafter b 0.0
Finished iteration 12111 of

(array([[1.26238926e-001, 9.85532611e-002, 1.90209755e-001],
        [7.34003139e-003, 9.28260888e-004, 4.41900447e-002],
        [9.42704241e-002, 1.13634187e-055, 9.55362763e-018],
        [4.16016324e-002, 2.51541146e-001, 1.33432651e-107],
        [2.28500703e-126, 3.22237254e-002, 1.78629669e-003],
        [8.04414948e-162, 2.31807327e-002, 9.49534236e-017],
        [2.21067730e-184, 3.85488630e-002, 1.67232754e-037],
        [1.77257302e-003, 9.17391286e-003, 1.06980657e-003],
        [1.41823646e-044, 1.59135436e-002, 1.18004758e-002],
        [7.92193520e-033, 1.33293567e-002, 3.86969452e-003],
        [8.92524200e-242, 3.75380308e-003, 9.14978786e-100],
        [1.83982412e-239, 5.44308766e-003, 4.22058363e-004],
        [2.79094083e-233, 4.47310710e-003, 1.14629132e-043],
        [2.36618184e-091, 4.19418238e-004, 2.31131267e-003],
        [5.61282760e-004, 6.81384465e-012, 3.19538026e-004],
        [3.58441366e-004, 7.83818565e-004, 6.11279522e-004],
        [5.75667773e-005

In [ ]:

filename = "Results/71GPa_subsolidus_mu3.0_eps1.0.hdf5"
# filename = "Results/aspims/aspim_bl_mu0.1_eps1.0_out.hdf5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a = list(f.keys())[0]
    b = list(f.keys())[1]
    p = list(f.keys())[2]

    # Get the data
    loaded_a = np.array(f[a])
    loaded_b = np.array(f[b])
    loaded_p = np.array(f[p])

h_spec = mdl.g_matr@loaded_p + loaded_b

f_b = "Results/0.3wt%Nd-Sm-Hf-Lu-U-A-X15-71GPa_subsolidus grains_01_Brg spectrum.hdf5"
f_c = "Results/0.3wt%Nd-Sm-Hf-Lu-U-A-X15-71GPa_subsolidus grains_01_CaPv spectrum.hdf5"
f_f = "Results/0.3wt%Nd-Sm-Hf-Lu-U-A-X15-71GPa_subsolidus grains_01_Fp spectrum.hdf5"

with h5py.File(f_b, "r") as f :
    brg = np.array(f["Experiments"]["EDSmap"]["data"])
with h5py.File(f_f, "r") as f :
    fp = np.array(f["Experiments"]["EDSmap"]["data"])
with h5py.File(f_c, "r") as f :
    capv = np.array(f["Experiments"]["EDSmap"]["data"])

x_true = np.linspace(-0.48,-0.48+1047*0.01,num=1047)
true_spectra = [brg,fp,capv]

In [ ]:
trunc_true = x_true > em.x[0]
trunc_res = (em.x < x_true[-1])
res_spec = [h_spec[:,0][trunc_res][:-1],h_spec[:,1][trunc_res][:-1],h_spec[:,2][trunc_res][:-1]]
true_spectra = [brg[trunc_true],fp[trunc_true],capv[trunc_true]]

angles=find_min_angle(true_spectra,res_spec)

print("Angle phase 0 :",angles[0])
print("Angle phase 1 :",angles[1])
print("Angle phase 2 :",angles[2])


In [ ]:
angles=find_min_angle(true_spectra,[h_spec[:,0],h_spec[:,1],h_spec[:,2]])

print("Angle phase 0 :",angles[0])
print("Angle phase 1 :",angles[1])
print("Angle phase 2 :",angles[2])


# Results

In [16]:
# Returns the angles between the ground truth and the endmembers found using SNMF
angles=find_min_angle(true_spectra,[mdl.get_phase_spectrum(0),mdl.get_phase_spectrum(1),mdl.get_phase_spectrum(2)])

maps=find_min_MSE(true_maps,[mdl.get_phase_map(0),mdl.get_phase_map(1),mdl.get_phase_map(2)])

print("Angle phase 0 :",angles[0])
print("Angle phase 1 :",angles[1])
print("Angle phase 2 :",angles[2])
print("MSE phase 0 :",maps[0])
print("MSE phase 1 :",maps[1])
print("MSE phase 2 :",maps[2])

# RITM0165818

Angle phase 0 : 8.80590593387548
Angle phase 1 : 9.237285139093306
Angle phase 2 : 7.154722215999662
MSE phase 0 : 174.40770434600745
MSE phase 1 : 101.81221325408632
MSE phase 2 : 47.92611403616286


### Visualisation of the results

In [19]:
# switch correspond to the index of the SNMF endmember
# true correspond to the index of the true spectrum
# The 2 should be changed independantly until a match is found
switch = 1
true= 1

plt.rcParams.update({'font.size': 22})
fig1 = plt.figure(figsize=(20, 12))
plt.subplot(121)
plt.plot(em.x,133*true_spectra[true]/np.sum(true_spectra[true]),'bo',label='truth',linewidth=4)
plt.plot(em.x, mdl.get_phase_spectrum(switch),'r-',label='reconstructed',markersize=3.5)
plt.legend(loc='best')
plt.xlim(0, 10)
plt.ylabel("Intensity")

plt.subplot(122)
plt.imshow(mdl.get_phase_map(switch), cmap="viridis")
plt.grid(b=30)
plt.title(f"Activations of first spectrum")
plt.colorbar()
plt.clim(0, 1)

fig1.tight_layout()

In [ ]:
# switch correspond to the index of the SNMF endmember
# true correspond to the index of the true spectrum
# The 2 should be changed independantly until a match is found
switch = 1
true=1

plt.rcParams.update({'font.size': 22})
fig1 = plt.figure(figsize=(20, 12))
plt.subplot(121)
plt.plot(em.x,true_spectra[true]/np.max(true_spectra[true]),'bo',label='truth',linewidth=4)
plt.plot(em.x, h_spec[:,switch]/np.max(h_spec[:,switch]),'r-',label='reconstructed',markersize=3.5)
plt.legend(loc='best')
plt.xlim(0, 10)
plt.ylabel("Intensity")

plt.subplot(122)
plt.imshow(loaded_a[switch].reshape(80,80), cmap="viridis")
plt.grid(b=30)
plt.title(f"Activations of first spectrum")
plt.colorbar()
plt.clim(0, 1)

fig1.tight_layout()

In [ ]:
# switch correspond to the index of the SNMF endmember
# true correspond to the index of the true spectrum
# The 2 should be changed independantly until a match is found
switch = 1
true=2

plt.rcParams.update({'font.size': 22})
fig1 = plt.figure(figsize=(20, 12))
plt.subplot(121)
plt.plot(x_true[trunc_true],50000*true_spectra[true]/np.sum(true_spectra[true]),'bo',label='truth',linewidth=4)
plt.plot(em.x,h_spec[:,switch],'r-',label='reconstructed',markersize=3.5)
plt.legend(loc='best')
plt.xlim(0, 10)
plt.ylabel("Intensity")

plt.subplot(122)
plt.imshow(loaded_a[switch].reshape(201,201), cmap="viridis")
plt.grid(b=30)
plt.title(f"Activations of first spectrum")
plt.colorbar()
plt.clim(0, 1)

fig1.tight_layout()

# Cross validation

# En chantier

In [ ]:
init_p_matr = np.random.rand(dl.g_matr.shape[1],dl.p_)
init_p_matr[:,0] = (np.linalg.inv(dl.g_matr.T@dl.g_matr)@dl.g_matr.T@(true_spectra[0]-1.9*Distributions.simplified_brstlg(Gaussians().x,b0,b1,b2,c0))).clip(min=1e-5)
init_p_matr[:,1] = (np.linalg.inv(dl.g_matr.T@dl.g_matr)@dl.g_matr.T@(true_spectra[1]-2.0 *Distributions.simplified_brstlg(Gaussians().x,b0,b1,b2,c0))).clip(min=1e-5)
init_p_matr[:,2] = (np.linalg.inv(dl.g_matr.T@dl.g_matr)@dl.g_matr.T@(true_spectra[2]-1.7*Distributions.simplified_brstlg(Gaussians().x,b0,b1,b2,c0))).clip(min=1e-5)

init_a_matr = np.random.rand(dl.p_, X.shape[0]*X.shape[1])
init_a_matr[0,:]= true_maps[0].reshape(6400)
init_a_matr[1,:]= true_maps[1].reshape(6400)
init_a_matr[2,:]= true_maps[2].reshape(6400)